In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


#from pycaret.regression import *
from sklearn.model_selection import train_test_split

#import lightgbm as lgb
import xgboost as xgb
import catboost as cb 

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_validate
import optuna

from optuna.samplers import TPESampler


In [2]:
def NMAE(true, pred):
    mae = np.mean(np.abs(np.exp(true)-np.round(np.exp(pred))))
    score = mae / np.mean(np.abs(np.exp(true)))
    return score

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [4]:
train[train['Target']>25]

,id,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Target
762,763,F,0.7,0.585,0.185,1.8075,0.7055,0.3215,0.475,29


In [5]:
def data_preprocess(df=None, test=None) :
    df = df.copy()
    
    # 부위별 무게 / 전체무게
    df['Shell Percent'] = df['Shell Weight']/df['Whole Weight']
    df['Viscra Percent'] = df['Viscra Weight']/df['Whole Weight']
    df['Shucked Percent'] = df['Shucked Weight']/df['Whole Weight']
    
    # (전체무게 - 각 부위무게) / 전체무게
    df['differ_weight_percent'] = abs(df['Shell Weight']+df['Shucked Weight']+df['Viscra Weight']-df['Whole Weight'])/df['Whole Weight']
    
    # 길이 비율
    df['ratio_1'] = df['Height']/df['Diameter']
    #df['ratio_2'] = df['Lenght']/df['Diameter']
    
    # 무게 비율 (부위/부위)
    #df['weight ratio_1'] =  (df['Shell Weight']+df['Viscra Weight']+df['Shucked Weight'])/df['Whole Weight']
    #df['weight ratio_2'] =  df['Shucked Weight']/df['Viscra Weight']
    #df['weight ratio_3'] =  df['Shell Weight']/df['Viscra Weight']
    #df['weight ratio_4'] =  df['Shell Weight']/df['Shucked Weight']
    
    # 3/30 쏘쏘....  
    #df['weight ratio_5'] =  (df['Shell Weight']+df['Shucked Weight'])/df['Viscra Weight']
    #df['weight ratio_6'] =  (df['Viscra Weight']+df['Shucked Weight'])/df['Shell Weight']
    #df['weight ratio_7'] =  (df['Viscra Weight']+df['Shell Weight'])/df['Shucked Weight']
    
    #df['weight ratio_8'] =  (df['Shucked Weight']+df['Shell Weight'])/df['Whole Weight']
    df['weight ratio_9'] =  (df['Viscra Weight']+df['Shucked Weight'])/df['Whole Weight']
    #df['weight ratio_10'] =  (df['Viscra Weight']+df['Shell Weight'])/df['Whole Weight']
    
    #gender를 0,1로 구분
    #df['Gender'] = df['Gender'].map(lambda x : 0 if x=='M' or x =="F" else 1)
    
    #cbrt
    #df['cbrt_mm'] = np.cbrt(df['Height']*df['Diameter']*df['Lenght'])
    
    
    if test == True : 
        
        return df 
    
    else :
        df = df.drop(762, axis=0)
        df['Target_log'] = np.log1p(df.Target)
   
        return df  

In [6]:
df_train = data_preprocess(train)

In [7]:
#Optuna용 train set

feature = ['Gender', 'Lenght', 'Diameter', 'Height', 
             'Whole Weight','Shucked Weight', 'Viscra Weight', 'Shell Weight',
             'Shell Percent', 'Viscra Percent', 'Shucked Percent', 
             'ratio_1', 'weight ratio_9',
             'differ_weight_percent']

X= df_train[feature]
y = df_train['Target_log']


In [8]:
def objective(trial):
    params = {
        #objective':'regression',
        #loss_function':'MAE',
        "random_state":42,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 300, 1000),
        "max_depth":trial.suggest_int("max_depth", 4, 12),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500)
        }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  
    cat_features =[0]
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid,y_valid)],
            early_stopping_rounds=35,cat_features=cat_features,
            verbose=100)
    cat_pred = cat.predict(X_valid)
    
    score = NMAE(y_valid, cat_pred)

    return score

In [9]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    direction = 'minimize',
    sampler = sampler,
    study_name='example-study',
    storage='sqlite:///example.db',
    load_if_exists=True)
study.optimize(objective, n_trials=10)


[I 2022-04-12 23:28:42,416] A new study created in RDB with name: example-study


0:	learn: 0.2717482	test: 0.2717482	test1: 0.3018934	best: 0.3018934 (0)	total: 185ms	remaining: 2m 30s
100:	learn: 0.1450203	test: 0.1495420	test1: 0.1911728	best: 0.1911728 (100)	total: 2.9s	remaining: 20.5s
200:	learn: 0.1247739	test: 0.1311154	test1: 0.1865226	best: 0.1863754 (188)	total: 5.85s	remaining: 17.8s


[I 2022-04-12 23:28:49,070] Trial 0 finished with value: 0.14773940904236385 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 813, 'max_depth': 9, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380}. Best is trial 0 with value: 0.14773940904236385.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.186375416
bestIteration = 188

Shrink model to first 189 iterations.
0:	learn: 0.2662509	test: 0.2662509	test1: 0.2748282	best: 0.2748282 (0)	total: 7.86ms	remaining: 7.69s


[I 2022-04-12 23:28:52,932] Trial 1 finished with value: 0.12955465587044535 and parameters: {'learning_rate': 0.11114989443094977, 'bagging_temperature': 0.012087541473056965, 'n_estimators': 979, 'max_depth': 11, 'random_strength': 21, 'colsample_bylevel': 0.5090949803242604, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357}. Best is trial 1 with value: 0.12955465587044535.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1559681884
bestIteration = 423

Shrink model to first 424 iterations.
0:	learn: 0.2857208	test: 0.2857208	test1: 0.2700867	best: 0.2700867 (0)	total: 731us	remaining: 581ms
100:	learn: 0.2042572	test: 0.2060264	test1: 0.2040360	best: 0.2040360 (100)	total: 2.75s	remaining: 18.9s
200:	learn: 0.1710247	test: 0.1740352	test1: 0.1821726	best: 0.1821726 (200)	total: 5.44s	remaining: 16.1s
300:	learn: 0.1553949	test: 0.1594970	test1: 0.1742025	best: 0.1742025 (300)	total: 8s	remaining: 13.1s
400:	learn: 0.1464608	test: 0.1516046	test1: 0.1708899	best: 0.1708899 (400)	total: 10.6s	remaining: 10.5s
500:	learn: 0.1405403	test: 0.1468080	test1: 0.1690905	best: 0.1690877 (499)	total: 13.5s	remaining: 7.95s
600:	learn: 0.1354216	test: 0.1423525	test1: 0.1683171	best: 0.1683066 (599)	total: 15.9s	remaining: 5.13s
700:	learn: 0.1313108	test: 0.1388001	test1: 0.1680900	best: 0.1680338 (688)	total: 18.3s	remaining: 2.45s


[I 2022-04-12 23:29:52,915] Trial 8 finished with value: 0.13507625272331153 and parameters: {'learning_rate': 0.010189592979395137, 'bagging_temperature': 18.27450885981603, 'n_estimators': 795, 'max_depth': 10, 'random_strength': 77, 'colsample_bylevel': 0.44442679104045424, 'l2_leaf_reg': 1.0760387199042735e-05, 'min_child_samples': 16, 'max_bin': 459}. Best is trial 3 with value: 0.11658698050753953.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1680337561
bestIteration = 688

Shrink model to first 689 iterations.
0:	learn: 0.2733207	test: 0.2732850	test1: 0.2641562	best: 0.2641562 (0)	total: 32ms	remaining: 11s
100:	learn: 0.1493641	test: 0.1514877	test1: 0.1754447	best: 0.1748364 (99)	total: 2.92s	remaining: 7.04s


[I 2022-04-12 23:29:57,164] Trial 9 finished with value: 0.13140794223826713 and parameters: {'learning_rate': 0.08330803890301997, 'bagging_temperature': 0.2106648601704221, 'n_estimators': 344, 'max_depth': 6, 'random_strength': 32, 'colsample_bylevel': 0.8377637070028385, 'l2_leaf_reg': 1.913034856594284e-05, 'min_child_samples': 90, 'max_bin': 342}. Best is trial 3 with value: 0.11658698050753953.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.174836441
bestIteration = 99

Shrink model to first 100 iterations.


In [122]:
feature = ['Gender', 'Lenght', 'Diameter', 'Height', 
             'Whole Weight','Shucked Weight', 'Viscra Weight', 'Shell Weight',
             'Shell Percent', 'Viscra Percent', 'Shucked Percent',              
             'differ_weight_percent']

X= df_train[feature]
y = df_train['Target_log']

In [123]:
sampler = TPESampler(seed=42)
study = optuna.create_study(direction = 'minimize',sampler = sampler)
study.optimize(objective, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-04-01 14:03:01,570] A new study created in memory with name: no-name-f63f6629-fa25-4912-b423-7896ac31a8f1


0:	learn: 0.3051154	test: 0.3051154	test1: 0.3395686	best: 0.3395686 (0)	total: 10.8ms	remaining: 8.81s
100:	learn: 0.1649111	test: 0.1682418	test1: 0.2092939	best: 0.2092939 (100)	total: 3.25s	remaining: 22.9s
200:	learn: 0.1474577	test: 0.1543148	test1: 0.2053402	best: 0.2052178 (177)	total: 6.31s	remaining: 19.2s


[I 2022-04-01 14:03:08,299] Trial 0 finished with value: 0.15953150242326333 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 813, 'max_depth': 9, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380}. Best is trial 0 with value: 0.15953150242326333.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.205217783
bestIteration = 177

Shrink model to first 178 iterations.
0:	learn: 0.3064307	test: 0.3064307	test1: 0.3036795	best: 0.3036795 (0)	total: 564us	remaining: 552ms
100:	learn: 0.1017845	test: 0.1163661	test1: 0.1893228	best: 0.1858891 (79)	total: 3.78s	remaining: 32.9s


[I 2022-04-01 14:03:12,948] Trial 1 finished with value: 0.14245585874799357 and parameters: {'learning_rate': 0.11114989443094977, 'bagging_temperature': 0.012087541473056965, 'n_estimators': 979, 'max_depth': 11, 'random_strength': 21, 'colsample_bylevel': 0.5090949803242604, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357}. Best is trial 1 with value: 0.14245585874799357.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1940956599
bestIteration = 315

Shrink model to first 316 iterations.
0:	learn: 0.3137252	test: 0.3136970	test1: 0.3117149	best: 0.3117149 (0)	total: 44.7ms	remaining: 35.5s
100:	learn: 0.2363348	test: 0.2377050	test1: 0.2455688	best: 0.2455688 (100)	total: 2.3s	remaining: 15.8s
200:	learn: 0.1926638	test: 0.1958310	test1: 0.2133148	best: 0.2133148 (200)	total: 5.2s	remaining: 15.4s
300:	learn: 0.1749736	test: 0.1795211	test1: 0.2029941	best: 0.2029941 (300)	total: 8.08s	remaining: 13.3s
400:	learn: 0.1650190	test: 0.1706240	test1: 0.1987526	best: 0.1987526 (400)	total: 11s	remaining: 10.8s
500:	learn: 0.1593845	test: 0.1657070	test1: 0.1968761	best: 0.1968760 (499)	total: 13.6s	remaining: 7.97s
600:	learn: 0.1546993	test: 0.1618556	test1: 0.1957212	best: 0.1957212 (600)	total: 16.2s	remaining: 5.23s
700:	learn: 0.1496029	test: 0.1578945	test1: 0.1948840	best: 0.1948840 (700)	total: 19.2s	remaining: 2.58s


[I 2022-04-01 14:04:11,838] Trial 8 finished with value: 0.1463610776035384 and parameters: {'learning_rate': 0.010189592979395137, 'bagging_temperature': 18.27450885981603, 'n_estimators': 795, 'max_depth': 10, 'random_strength': 77, 'colsample_bylevel': 0.44442679104045424, 'l2_leaf_reg': 1.0760387199042735e-05, 'min_child_samples': 16, 'max_bin': 459}. Best is trial 5 with value: 0.12921125644074513.


794:	learn: 0.1460000	test: 0.1552197	test1: 0.1943359	best: 0.1943359 (794)	total: 21.6s	remaining: 0us

bestTest = 0.1943359309
bestIteration = 794

0:	learn: 0.3059725	test: 0.3059725	test1: 0.2797779	best: 0.2797779 (0)	total: 39.9ms	remaining: 13.7s
100:	learn: 0.1665384	test: 0.1690093	test1: 0.1939589	best: 0.1939589 (100)	total: 3.16s	remaining: 7.6s


[I 2022-04-01 14:04:17,417] Trial 9 finished with value: 0.1440781440781441 and parameters: {'learning_rate': 0.08330803890301997, 'bagging_temperature': 0.2106648601704221, 'n_estimators': 344, 'max_depth': 6, 'random_strength': 32, 'colsample_bylevel': 0.8377637070028385, 'l2_leaf_reg': 1.913034856594284e-05, 'min_child_samples': 90, 'max_bin': 342}. Best is trial 5 with value: 0.12921125644074513.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1938011161
bestIteration = 125

Shrink model to first 126 iterations.
Best Score: 0.12921125644074513
Best trial {'learning_rate': 0.09519754482692679, 'bagging_temperature': 0.17654048052495078, 'n_estimators': 664, 'max_depth': 8, 'random_strength': 18, 'colsample_bylevel': 0.9817507766587351, 'l2_leaf_reg': 2.3256233372599825e-05, 'min_child_samples': 95, 'max_bin': 469}


In [17]:
from sklearn.model_selection import KFold

In [18]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
folds = []

In [19]:
for train_idx, valid_idx in kfold.split(X, y):
    folds.append((train_idx, valid_idx))

In [20]:
cat_models={}

cat_features =[0]

for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = X.iloc[train_idx].values 
    X_valid = X.iloc[valid_idx].values
    y_train = y.iloc[train_idx].values
    y_valid = y.iloc[valid_idx].values

    cat = CatBoostRegressor(**study.best_params)
    cat.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid,y_valid)],
            early_stopping_rounds=35,cat_features=cat_features,
            verbose=100)
    cat_models[fold] = cat
    print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 0.3067776	test: 0.3067776	test1: 0.3041877	best: 0.3041877 (0)	total: 56.7ms	remaining: 18.8s
100:	learn: 0.1508475	test: 0.1574038	test1: 0.1919944	best: 0.1913611 (77)	total: 3.5s	remaining: 8.01s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1913611287
bestIteration = 77

Shrink model to first 78 iterations.


====================================2============================================
0:	learn: 0.3148633	test: 0.3148633	test1: 0.3092885	best: 0.3092885 (0)	total: 3.51ms	remaining: 1.16s
100:	learn: 0.1538524	test: 0.1600203	test1: 0.1826653	best: 0.1824938 (92)	total: 3.64s	remaining: 8.33s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1824938178
bestIteration = 92

Shrink model to first 93 iterations.


====================================3============================================
0:	learn: 0.3129336	test: 0.3129336	test1: 0.3000019	best: 

In [21]:
submit_test = pd.read_csv('data/sample_submission.csv')

In [22]:
## test데이터셋 5fold 학습하여 평균 
df_test = data_preprocess(test,True)
df_test[feature]

,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Shell Percent,Viscra Percent,Shucked Percent,ratio_1,weight ratio_9,differ_weight_percent
0,F,0.595,0.470,0.155,1.1210,0.4515,0.1780,0.1550,0.138269,0.158787,0.402765,0.329787,0.561552,0.300178
1,M,0.580,0.450,0.150,0.9270,0.2760,0.1815,0.3600,0.388350,0.195793,0.297735,0.333333,0.493528,0.118123
2,I,0.260,0.205,0.070,0.0970,0.0415,0.0190,0.0305,0.314433,0.195876,0.427835,0.341463,0.623711,0.061856
3,M,0.590,0.460,0.130,1.1020,0.4550,0.2055,0.3300,0.299456,0.186479,0.412886,0.282609,0.599365,0.101180
4,F,0.595,0.465,0.140,1.1130,0.5175,0.2440,0.3050,0.274034,0.219227,0.464960,0.301075,0.684187,0.041779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,I,0.170,0.105,0.035,0.0340,0.0120,0.0085,0.0050,0.147059,0.250000,0.352941,0.333333,0.602941,0.250000
2920,I,0.435,0.345,0.115,0.4180,0.2220,0.0735,0.1060,0.253589,0.175837,0.531100,0.333333,0.706938,0.039474
2921,I,0.570,0.450,0.135,0.7940,0.3815,0.1415,0.2450,0.308564,0.178212,0.480479,0.300000,0.658690,0.032746
2922,I,0.460,0.350,0.120,0.4885,0.1930,0.1050,0.1550,0.317298,0.214944,0.395087,0.342857,0.610031,0.072671


In [23]:
submit_test.iloc[:,1:]=0

for fold in range(5) : 
    submit_test.iloc[:,1] += cat_models[fold].predict(df_test[feature])/5

In [24]:
submit_test

,id,Target
0,1,2.363000
1,2,2.597812
2,3,1.679858
3,4,2.426255
4,5,2.267091
...,...,...
2919,2920,1.471561
2920,2921,1.944186
2921,2922,2.227234
2922,2923,2.198858
